In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from scipy import fftpack

In [ ]:
# 自定义二维傅里叶变换（性能过差）
def my_fftn(img):
    h, w = img.shape
    F = np.zeros((h,w), dtype=complex)
    # 计算傅里叶变换
    for u in range(h):
        for v in range(w):
            for x in range(h):
                for y in range(w):
                    F[u,v]+=img[x,y]*np.exp(-2j*np.pi*((u*x/h)+(v*y/w)))
    return F

In [ ]:
# 改进（自定义FFT）
# 自定义的一维fft，使用Cooley-Tukey递归算法
def fft1d(x):
    N = len(x)
    if N<=1:
        return x
    even = fft1d(x[0::2])
    odd = fft1d(x[1::2])
    T = [np.exp(-2j*np.pi*k/N)*odd[k] for k in range(N//2)]
    return [even[k]+T[k] for k in range(N//2)]+[even[k]-T[k] for k in range(N//2)]
# 自定义的二维fft
def fft2d(x):
    temp = np.apply_along_axis(fft1d,1,x)
    result = np.apply_along_axis(fft1d,0,temp)
    return result

In [ ]:
# 自定义逆fft
def ifft1d(x):
    N = len(x)
    if N <= 1:
        return x
    even = ifft1d(x[0::2])
    odd = ifft1d(x[1::2])
    T = [np.exp(2j * np.pi * k / N) * odd[k] for k in range(N // 2)]
    return [(even[k] + T[k]) / 2 for k in range(N // 2)] + [(even[k] - T[k]) / 2 for k in range(N // 2)]

def ifft2d(x):
    temp = np.apply_along_axis(ifft1d, 1, x)
    result = np.apply_along_axis(ifft1d, 0, temp)
    return result

In [ ]:
# 频移
def my_fftshift(F):
    h,w =F.shape
    cx, cy=h//2, w//2
    shift_F = np.block([[F[cx:,cy:],F[cx:,:cy]],[F[:cx,cy:],F[:cx,:cy]]])
    return shift_F

In [ ]:
# 打开图片
img1 = Image.open('lena-512-gray.png').convert('L')
img1_array=np.array(img1)

In [ ]:
# 自己实现的傅里叶变换,并使用频移获得频谱图
F1 = fft2d(img1_array)
F1_magnitude = np.abs(my_fftshift(F1))

In [ ]:
# 用fft-pack的内置函数
F2 = fftpack.fftn(img1_array)
F2_magnitude = np.abs(fftpack.fftshift(F2))

In [ ]:
# 在1e-8的精度下比较
assert np.allclose(F1, F2, atol=1e-8,rtol=0),"not equal"
assert np.allclose(F1_magnitude, F2_magnitude, atol=1e-8,rtol=0),"not equal"
# 保存频谱图
# 对数变换，使值域变为正值
F1_magnitude_log = np.log1p(F1_magnitude)

# 归一化到0-255
F1_magnitude_rescale = 255 * (F1_magnitude_log - F1_magnitude_log.min()) / (F1_magnitude_log.max() - F1_magnitude_log.min())

# 转为8位无符号整数
F_magnitude_uint8 = np.uint8(F1_magnitude_rescale)

# 转为Pillow图像并保存
Image.fromarray(F_magnitude_uint8).save('lena-512-freq.png')

In [ ]:
# 定义滤波器
# 理想低通滤波器
def ideal_lowpass_filter(img,cutoff):
    rows, cols=img.shape
    crow, ccol=rows//2,cols//2
    mask = np.zeros((rows,cols))
    for i in range(rows):
        for j in range(cols):
            dist = np.sqrt((i-crow)**2+(j-ccol)**2)
            if dist<=cutoff:
                mask[i,j]=1
    return mask

# 理想高通滤波器
def ideal_highpass_filter(img,cutoff):
    rows, cols=img.shape
    crow, ccol=rows//2,cols//2
    mask = np.zeros((rows,cols))
    for i in range(rows):
        for j in range(cols):
            dist = np.sqrt((i-crow)**2+(j-ccol)**2)
            if dist>=cutoff:
                mask[i,j]=1
    return mask

In [ ]:
# 使用高通滤波器
cutoff=50
mask_highpass = ideal_highpass_filter(img1_array,cutoff=cutoff)
F_filtered_highpass = F1*my_fftshift(mask_highpass)
img_filtered_highpass_1=np.abs(ifft2d(F_filtered_highpass))
img_filtered_highpass_2=np.abs(fftpack.ifftn(F_filtered_highpass))

In [ ]:
assert np.allclose(img_filtered_highpass_1,img_filtered_highpass_2,atol=1e-8,rtol=0),"not equal"

In [ ]:
img_highpass = Image.fromarray(img_filtered_highpass_1.astype('uint8'))
img_highpass.show()
img_highpass.save('lena-512-highpass-' + str(cutoff) + '.png')

In [ ]:
# 绘制频率响应
def plot_horizontal_slice(mask, title='Horizontal Slice through the Center'):
    center_row = mask.shape[0] // 2
    plt.figure()
    plt.plot(mask[center_row, :])
    plt.title(title)
    plt.xlabel('Frequency')
    plt.ylabel('Amplitude')
    plt.grid(True)
    plt.show()

# Example usage:
img_shape = (256, 256)  # Example image shape

In [2]:
# 频率响应
cutoff_low = 20
cutoff_high = 50

# Generate the masks
lowpass_mask = ideal_lowpass_filter(np.zeros(img_shape), cutoff_low, order)

# Plot horizontal and vertical slices
plot_horizontal_slice(lowpass_mask, title='Horizontal Slice of Butterworth Low-pass Filter')

NameError: name 'ideal_lowpass_filter' is not defined